In [1]:
# Add the parent directory to path
import sys

sys.path.append("..")

In [45]:
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np

import json

In [29]:
q3a_folder = Path("../results/q3a")
q3b_folder = Path("../results/q3b")
q4a_folder = Path("../results/q4a")

visualization_folder = Path("../results/visualization")
visualization_folder.mkdir(exist_ok=True, parents=True)

## Q3a Plots

In [37]:
def get_flattened_data(run_folder: list[Path]) -> dict[str, np.ndarray]:
    files = run_folder.glob("*.json")
    run_ids_paths = [(int(str(x).split("_")[2]), x) for x in files]
    data = {}
    for run_id, run_path in run_ids_paths:
        with open(run_path, "r") as f:
            data[run_id] = json.load(f)
            
    flattened_data = {}
    for run_id, run_data in data.items():
        for key, value in run_data.items():
            if key in flattened_data:
                flattened_data[key].append(value)
            else:
                flattened_data[key] = [value]


    for key, value in flattened_data.items():
        flattened_data[key] = np.stack(value)
        print(key, flattened_data[key].shape)
    
    return flattened_data

In [144]:
def visualize_run_metrics(run_folder: Path, run_title: str):
    flattened_data = get_flattened_data(run_folder)
    
    # Plot the average reward
    df = pd.DataFrame(flattened_data["eval_trace_mean"], index = ["run_1", "run_2", "run_3"]).transpose()
    df["eval_step"] = np.arange(1, len(df) + 1)
    t = df.melt(id_vars="eval_step", value_name="eval_reward")[["eval_step", "eval_reward"]]
    plt = sns.lineplot(data=t, x="eval_step", y="eval_reward", label="Average reward")
    plt.set_title(f"{run_title}: Average evaluation reward (averaged over 3 runs)")
    plt.set_xlabel("Evaluation step")
    plt.set_ylabel("Reward")
    plt.hlines(max_val := t["eval_reward"].max(), xmin=0, xmax=t["eval_step"].max(), linestyles="dashed", label=f"Maximum value ({max_val:.2f})", color="black", alpha=0.5)
    plt.legend(loc="lower right")
    plt.figure.savefig(visualization_folder / f"{run_title}_eval_reward.png", dpi=300)
    plt.figure.clf()
    
    
    # Plot the training rewards
    df = pd.DataFrame(flattened_data["tr_trace"], index = ["run_1", "run_2", "run_3"]).transpose()
    df["tr_step"] = np.arange(1, len(df) + 1)
    t = df.melt(id_vars="tr_step", value_name="tr_reward")[["tr_step", "tr_reward"]]
    plt = sns.lineplot(data=t, x="tr_step", y="tr_reward", label="Training reward")
    plt.set_title(f"{run_title}: Training reward (averaged over 3 runs)")
    plt.set_xlabel("Training step")
    plt.set_ylabel("Reward")
    plt.hlines(max_val := t["tr_reward"].max(), xmin=0, xmax=t["tr_step"].max(), linestyles="dashed", label=f"Maximum value ({max_val:.2f})", color="black", alpha=0.5)
    plt.legend(loc="lower right")
    plt.figure.savefig(visualization_folder / f"{run_title}_tr_reward.png", dpi=300)
    plt.figure.clf()
    
    # Plot the training losses
    df = pd.DataFrame(flattened_data["tr_losses"], index = ["run_1", "run_2", "run_3"]).transpose()
    df["tr_step"] = np.arange(1, len(df) + 1)
    t = df.melt(id_vars="tr_step", value_name="tr_loss")[["tr_loss", "tr_step"]]
    plt = sns.lineplot(data=t, x="tr_step", y="tr_loss", label="Training loss")
    plt.set_title(f"{run_title}: Training loss (averaged over 3 runs)")
    plt.set_xlabel("Training step")
    plt.set_ylabel("Loss")
    plt.legend(loc="upper right")
    plt.figure.savefig(visualization_folder / f"{run_title}_tr_loss.png", dpi=300)
    plt.figure.clf()
# Delete the items inside the visualization folder
for item in visualization_folder.glob("*"):
    item.unlink()
    
visualize_run_metrics(q3a_folder, "3-1")

tr_trace (3, 500)
eval_trace_mean (3, 26)
eval_trace_std (3, 26)
tr_losses (3, 12953)
eval_seed_sequences (3, 20)
tr_trace (3, 500)
eval_trace_mean (3, 26)
eval_trace_std (3, 26)
tr_losses (3, 12953)
eval_seed_sequences (3, 20)
tr_trace (3, 600)
eval_trace_mean (3, 31)
eval_trace_std (3, 31)
tr_losses (3, 15953)
eval_seed_sequences (3, 20)
learning_rates (3, 15953)


<Figure size 640x480 with 0 Axes>

In [ ]:
visualize_run_metrics(q3b_folder, "3-2")

In [ ]:
visualize_run_metrics(q4a_folder, "4-1")